In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

In [2]:
plik = open("test.txt")

slowka = [slowo.lower().replace("\n", "") for index,slowo in enumerate(plik)]
slowa = []

print(len(slowka))

# while len(slowa) < 1000:
#     los = random.randint(0, len(slowka) - 1)
#     if slowka[los] not in slowa:
#         slowa.append(slowka[los])

# print(len(slowa))
# print(slowa[0:200])

# slowa = slowa[0:2000]
slowa = slowka

400


In [3]:
p1 = open("test.txt", "a")
for slowo in slowa:
    p1.write(slowo+"\n")
p1.close()

In [3]:
plikembd = open("../character_embeddings.txt")

embeddings = {}

for line in plikembd.readlines():
    exp1 = line.replace("\n", "").split(":")
    
    znak = exp1[0]
    embedding = [float(liczba) for liczba in exp1[1].replace(" ", "").replace("[", "").replace("]", "").split(",\t")[:-1]]

    
    embeddings[znak] = embedding
    
embeddings['🈳'] = [0 for __ in range(1600)]

In [4]:
def podobienstwo_cosinusowe(wektor1, wektor2, podobienstwo_min=0.01):
    if np.sum(wektor1) == 0 or np.sum(wektor2) == 0:
        return 0
    
    sumaup = np.sum([(wektor1[x] * wektor2[x]) for x in range(len(wektor1))])
    s1 = np.sum([skladowa**2 for skladowa in wektor1])
    s2 = np.sum([skladowa**2 for skladowa in wektor2])
    
    down = math.sqrt(s1)*math.sqrt(s2)
    
    outt = sumaup/down
    
    if outt>=podobienstwo_min:
        return outt
    else:
        return 0 
    
podobienstwo_cosinusowe(embeddings['a'], embeddings['🈳'])

0

In [5]:
def znajdz_najblizszy(wektor, embeddings):
    najblizszy = '🈳'
    podobienstwo = 0
    
    for key in embeddings:
        pdb = podobienstwo_cosinusowe(wektor, embeddings[key])
        
        if pdb > podobienstwo:
            podobienstwo = pdb
            najblizszy = key
    return najblizszy, podobienstwo
    
znajdz_najblizszy(np.random.rand(1600), embeddings)

('@', 0.0750612887457633)

In [6]:
def build_slowo(slowo, embeddings, dlugosc):
    zwrot = np.array([])
    for literka in slowo:
        zwrot = np.append(zwrot, embeddings[literka])
    
    roznica = int(dlugosc - len(zwrot)/len(embeddings[list(embeddings.keys())[0]]))
    
    for __ in range(roznica):
        zwrot = np.append(zwrot, [0 for __ in range(len(embeddings[literka]))])
    
    return zwrot.reshape(-1)

build_slowo("a", embeddings, 7).shape

(11200,)

In [7]:
def najdluzsze_slowo(slowa):
    najdl = 0
    
    for slowo in slowa:
        if len(slowo) > najdl:
            najdl = len(slowo)
            
    return najdl

longest = 16
print(longest)

16


In [8]:
def przetlumacz_wyjscia(wyjscia_flat, embeddings):
    dlugosc = len(embeddings[list(embeddings.keys())[0]])
    
    wyjscia = wyjscia_flat.reshape(-1, dlugosc)
    
    zwrot = ""
    
    for wyjscie in wyjscia:
        zwrot = zwrot + znajdz_najblizszy(wyjscie, embeddings)[0]
        
    return zwrot

In [9]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.conv1 = nn.ConvTranspose1d(1, 4, 3, 1, 1)
        self.mp1 =  nn.MaxPool1d(4)
        
        self.conv2 = nn.ConvTranspose1d(4, 8, 5, 1, 2)
        self.mp2 = nn.MaxPool1d(8)
        
        
        self.fc_in = nn.Linear(6400, 3000)
        self.fc_h1 = nn.Linear(3000, 2000)
        self.fc_h2 = nn.Linear(2000, 1000)
        self.fc_out = nn.Linear(1000, 200)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.mp1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = self.mp2(x)
        x = F.relu(x)
        
        x = x.reshape(len(x), -1)
        
        x = self.fc_in(x)
        x = F.relu(x)
        
        x = self.fc_h1(x)
        x = F.relu(x)
        
        x = self.fc_h2(x)
        x = F.relu(x)
        
        x = self.fc_out(x)
        x = F.relu(x)
        
        return x

In [10]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc_1 = nn.Linear(500, 1500)
        self.fc_out = nn.Linear(1500, 1800)
        
        self.conv1 = nn.ConvTranspose1d(100, 32, 8, 8, 0)
        self.norm1 =  nn.BatchNorm1d(32)
        
        self.conv2 = nn.ConvTranspose1d(32, 16, 6, 6, 4)
        self.norm2 =  nn.BatchNorm1d(16)
      
        self.conv3 = nn.ConvTranspose1d(16, 4, 6, 6, 6)
        self.norm3 =  nn.BatchNorm1d(4)
        
        self.conv4 = nn.ConvTranspose1d(4, 1, 5, 5, 10)
        self.norm4 =  nn.BatchNorm1d(1)
        
    def forward(self, x):
        x = self.fc_1(x)
        x = F.relu(x)
        
        x = self.fc_out(x)
        x = F.relu(x)
        
        x = x.reshape(len(x), 100, -1)
        
        x = self.conv1(x)
        x = self.norm1(x)
        
        x = self.conv2(x)
        x = self.norm2(x)
        
        x = self.conv3(x)
        x = self.norm3(x)
        
        x = self.conv4(x)
        #x = self.norm4(x)
        
        return x

In [11]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        #nn.init.xavier_normal(m.weight.data, 1.0, 0.002)
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [12]:
classifier = Discriminator().to("cuda:0")
generator = Generator().to("cuda:0")

In [13]:
generator.apply(weights_init)
classifier.apply(weights_init)

Discriminator(
  (conv1): ConvTranspose1d(1, 4, kernel_size=(3,), stride=(1,), padding=(1,))
  (mp1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): ConvTranspose1d(4, 8, kernel_size=(5,), stride=(1,), padding=(2,))
  (mp2): MaxPool1d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
  (fc_in): Linear(in_features=6400, out_features=3000, bias=True)
  (fc_h1): Linear(in_features=3000, out_features=2000, bias=True)
  (fc_h2): Linear(in_features=2000, out_features=1000, bias=True)
  (fc_out): Linear(in_features=1000, out_features=200, bias=True)
)

In [14]:
TensorFakeSeeds = torch.rand(len(TensorRealIns), 500).to("cuda:0")
TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")

print(TensorFakeIns.shape)

NameError: name 'TensorRealIns' is not defined

In [14]:
manualSeed = 1001
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [15]:
TensorRealOuts = torch.ones((len(slowa), 200)).float().to("cuda:0")
TensorFakeOuts = torch.zeros((len(slowa), 200)).float().to("cuda:0")

TensorOuts = torch.cat((TensorRealOuts, TensorFakeOuts), 0)

print(TensorRealOuts.shape)
print(TensorFakeOuts.shape)

torch.Size([400, 200])
torch.Size([400, 200])


In [16]:
criterion = nn.BCELoss()

In [17]:
optimizerGenerator = optim.Adam(generator.parameters(), lr=1e-3)
optimizerClassifier = optim.Adam(classifier.parameters(), lr=1e-6)

print(optimizerGenerator)
print(optimizerClassifier)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-06
    weight_decay: 0
)


In [18]:
probki = np.array([])

for slowo in slowa:
    probki = np.append(probki, build_slowo(slowo, embeddings, longest))
    
probki = probki.reshape(len(slowa), 1, -1)

TensorRealIns = torch.tensor(probki).float().to("cuda:0")

print(TensorRealIns.shape)

torch.Size([400, 1, 25600])


In [19]:
GeneratorLosses = []
ClassifierLosses = []

In [25]:
for epoch in range(1000):
    TensorFakeSeeds = torch.rand(len(TensorRealIns), 500).to("cuda:0")
    TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")
    
    if epoch == 1:
        print(TensorFakeIns.shape)
    
    TensorIns = torch.cat((TensorRealIns, TensorFakeIns.detach()), 0)
    
    classifier.train()
    optimizerClassifier.zero_grad()
    
    y_ = classifier(TensorIns)
    lossClassifier = criterion(y_, TensorOuts)
    
    lossClassifier.backward()
    optimizerClassifier.step()
    
    generator.train()
    optimizerGenerator.zero_grad()
    
    y_ = classifier(TensorFakeIns)
    lossGenerator = criterion(y_, TensorRealOuts)
    
    lossGenerator.backward()
    optimizerGenerator.step()
    
    GeneratorLosses.append(lossGenerator)
    ClassifierLosses.append(lossClassifier)
    
    if epoch % 1000 == 0:
        print(lossClassifier, lossGenerator)
        print(przetlumacz_wyjscia(TensorFakeIns[0].cpu().detach().numpy(), embeddings))

tensor(6.4583, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) tensor(12.9305, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
RhPon`@y.@#i,g`@
torch.Size([400, 1, 25600])


In [26]:
fix, ax = plt.subplots()
ax.plot([x for x in range(len(GeneratorLosses))], GeneratorLosses, ".", [x for x in range(len(GeneratorLosses))], ClassifierLosses, ".")

FigureCanvasNbAgg()

In [28]:
TensorFakeSeeds = torch.rand(5, 500).to("cuda:0")
TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")


outs = TensorFakeIns.cpu().detach().numpy()
for out in outs:
    print(przetlumacz_wyjscia(out, embeddings))

L5K@@@@ww@oi^5`@
L5K@@@@ww@oi^5`@
L5K^@@@ww@oi^5`@
L5K@@@@ww@oi^5`@
L5K^@@@ww@oi^5`@
